# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file=os.path.join('..','WeatherPy','output_data','Clean_City_data.csv')
df=pd.read_csv(file)
del df['Unnamed: 0']
df.head()


,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date
0,nuuk,64.18,-51.72,35.31,93.0,100.0,13.87,GL,1.592281e+09
1,staryy nadym,65.61,72.68,55.96,80.0,100.0,17.69,RU,1.592281e+09
2,avarua,-21.21,-159.78,71.60,60.0,61.0,5.82,CK,1.592280e+09
3,yarmouth,41.71,-70.23,55.65,87.0,1.0,10.89,US,1.592281e+09
4,rikitea,-23.12,-134.97,75.63,83.0,100.0,9.86,PF,1.592280e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]
humidity=df["Humidity (%)"]

# Plot Heatmap
fig = gmaps.figure(map_type='TERRAIN', center=(0, 0), zoom_level=2,layout={
        'width': '1000px',
        'height': '700px',
        'padding': '3px',
        'border': '1px solid black'
})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig



Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df=df.loc[(df["Max Temp (F)"]>=70) & (df["Max Temp (F)"]<=80) & (df["Wind Speed"]<=10) & (df["Cloudiness (%)"]==0),:]
ideal_df=ideal_df.reset_index(drop=True)
ideal_df.head(20)

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date
0,san patricio,19.22,-104.70,78.53,73.0,0.0,5.03,MX,1.592281e+09
1,inhuma,-6.67,-41.71,70.68,83.0,0.0,7.96,BR,1.592281e+09
2,takab,36.40,47.11,70.72,29.0,0.0,3.71,IR,1.592281e+09
3,floriano,-6.77,-43.02,73.60,72.0,0.0,4.21,BR,1.592281e+09
4,sao raimundo nonato,-9.02,-42.70,71.69,70.0,0.0,5.46,BR,1.592281e+09
5,neka,36.65,53.30,75.20,69.0,0.0,6.93,IR,1.592281e+09
6,lago da pedra,-4.33,-45.17,72.14,95.0,0.0,2.19,BR,1.592281e+09
7,porto nacional,-10.71,-48.42,73.40,78.0,0.0,1.12,BR,1.592281e+09
8,santa maria del oro,25.93,-105.37,74.08,26.0,0.0,6.80,MX,1.592281e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotels=[]

# Loop through dataframe to get nearest hotel
for city in range(len(ideal_df["City"])):
    lat = ideal_df.loc[city]["Lat"]
    lng = ideal_df.loc[city]["Lng"]
    city_coords = f"{lat},{lng}"
    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
              }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_response = requests.get(base_url, params=params).json()
 
    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
ideal_df["Nearest Hotel"] = hotels
ideal_df.head(10)



,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date,Nearest Hotel
0,san patricio,19.22,-104.70,78.53,73.0,0.0,5.03,MX,1.592281e+09,Hotel Cabo Blanco
1,inhuma,-6.67,-41.71,70.68,83.0,0.0,7.96,BR,1.592281e+09,Yonahanakbar
2,takab,36.40,47.11,70.72,29.0,0.0,3.71,IR,1.592281e+09,مهمانپذیر تخت سلیمان
3,floriano,-6.77,-43.02,73.60,72.0,0.0,4.21,BR,1.592281e+09,Hotel Maktub
4,sao raimundo nonato,-9.02,-42.70,71.69,70.0,0.0,5.46,BR,1.592281e+09,Real Hotel
5,neka,36.65,53.30,75.20,69.0,0.0,6.93,IR,1.592281e+09,برق خودرو کولر مجید
6,lago da pedra,-4.33,-45.17,72.14,95.0,0.0,2.19,BR,1.592281e+09,Nearest hotel not found
7,porto nacional,-10.71,-48.42,73.40,78.0,0.0,1.12,BR,1.592281e+09,Vila Real Hotel
8,santa maria del oro,25.93,-105.37,74.08,26.0,0.0,6.80,MX,1.592281e+09,Hotel Palacio Del Sol


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]


In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1000px'))